In [1]:
import tensorflow as tf
import os
import pickle
import numpy as np

CIFAR_DIR = "./cifar-10-batches-py"
print(os.listdir(CIFAR_DIR))

/home/wp/.virtualenvs/tf1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/wp/.virtualenvs/tf1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/wp/.virtualenvs/tf1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/wp/.virtualenvs/tf1/lib/python3.6/site-packages/tensorflow/python/f

['data_batch_3', 'data_batch_2', 'readme.html', 'data_batch_4', 'batches.meta', 'data_batch_1', 'data_batch_5', 'test_batch']


In [2]:
# 和前面的vgg是一致的
def load_data(filename):
    """read data from data file."""
    with open(filename, 'rb') as f:
        data = pickle.load(f, encoding='bytes')
        return data[b'data'], data[b'labels']

# tensorflow.Dataset.
class CifarData:
    def __init__(self, filenames, need_shuffle):
        all_data = []
        all_labels = []
        for filename in filenames:
            data, labels = load_data(filename)
            all_data.append(data)
            all_labels.append(labels)
        self._data = np.vstack(all_data)
        self._data = self._data / 127.5 - 1
        self._labels = np.hstack(all_labels)
        print(self._data.shape)
        print(self._labels.shape)
        
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0
        if self._need_shuffle:
            self._shuffle_data()
            
    def _shuffle_data(self):
        # [0,1,2,3,4,5] -> [5,3,2,4,0,1]
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
        self._labels = self._labels[p]
    
    def next_batch(self, batch_size):
        """return batch_size examples as a batch."""
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception("have no more examples")
        if end_indicator > self._num_examples:
            raise Exception("batch size is larger than all examples")
        batch_data = self._data[self._indicator: end_indicator]
        batch_labels = self._labels[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_labels

train_filenames = [os.path.join(CIFAR_DIR, 'data_batch_%d' % i) for i in range(1, 6)]
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]

train_data = CifarData(train_filenames, True)
test_data = CifarData(test_filenames, False)

(50000, 3072)
(50000,)
(10000, 3072)
(10000,)


In [3]:
# 下面是残差块，因为resnet是一个一个残差块
def residual_block(x, output_channel):
    """residual connection implementation"""
    input_channel = x.get_shape().as_list()[-1]  # 这个是输入通道数
    
    # 如果输出通道是输入通道的两倍，因为输出通道的数目增加了，所以需要增加维度，increase_dim = True
    if input_channel * 2 == output_channel:  
        increase_dim = True
        strides = (2, 2)
    elif input_channel == output_channel:  # 如果输入通道和输出通道一致，图像大小不变,increase_dim是False
        increase_dim = False
        strides = (1, 1)
    else:
        raise Exception("input channel can't match output channel")
    
    # 接着根据图形经过两个卷积层，经过两个卷积层，输出大小变为1半，大小和通道数是两码事
    conv1 = tf.layers.conv2d(x,
                             output_channel,
                             (3,3),
                             strides = strides,
                             padding = 'same',
                             activation = tf.nn.relu,
                             name = 'conv1')
    conv2 = tf.layers.conv2d(conv1,
                             output_channel,
                             (3, 3),
                             strides = (1, 1),
                             padding = 'same',
                             activation = tf.nn.relu,
                             name = 'conv2')
    
    if increase_dim:  # 如果通道数增加，就做平均值池化
        # [None, image_width, image_height, channel] -> [,,,channel*2]
        pooled_x = tf.layers.average_pooling2d(x,
                                               (2, 2),
                                               (2, 2),
                                               padding = 'valid')  # 这里平均值和最大值池化都可以选
        # 因为前面3维是样本数，宽，高，维度不变，最后一维通道左右各增加一半，比如原来通道是32，现在再32，就是64
        # 就可以相加(pad的作用)
        padded_x = tf.pad(pooled_x,
                          [[0,0],
                           [0,0],
                           [0,0],
                           [input_channel // 2, input_channel // 2]]) 
    else:
        padded_x = x  # 不increase_dim，保持最初不变，直接相加
    output_x = conv2 + padded_x  # 这里就是残差连接的基本结构，padded_x就是图中直连的那条线（恒等映射）
    return output_x

# 再通过残差块构建res_net模型
def res_net(x, 
            num_residual_blocks, 
            num_filter_base,
            class_num):
    """residual network implementation"""
    """
    Args:
    - x:
    - num_residual_blocks: eg: [3, 4, 6, 3] 每一层的残差块数目
    - num_filter_base:初始通道数
    - class_num:分类数
    """
    num_subsampling = len(num_residual_blocks)  # 需要做多少次降采样
    layers = []  # 后面依次把层添加到里边
    # x: [None, width, height, channel] -> [width, height, channel]
    input_size = x.get_shape().as_list()[1:]  #这里得到的是32*32*3
    with tf.variable_scope('conv0'):
        conv0 = tf.layers.conv2d(x, 
                                 num_filter_base,
                                 (3, 3),
                                 strides = (1, 1),
                                 padding = 'same',
                                 activation = tf.nn.relu,
                                 name = 'conv0')  # 和图中类似，首先经过一个普通卷积
        layers.append(conv0)
    
    # 下面才开始放残差块
    # eg:num_subsampling = 4, sample_id = [0,1,2,3]
    for sample_id in range(num_subsampling):  # 有几组残差块
        for i in range(num_residual_blocks[sample_id]):  # 每组中去构建几个残差块
            with tf.variable_scope("conv%d_%d" % (sample_id, i)):
                conv = residual_block(
                    layers[-1],  # 每次拿最新的层次的输出 作为下一层的输入
                    num_filter_base * (2 ** sample_id))  # 2的零次方是1，通道数是不断翻倍的过程
                layers.append(conv)  # 帮刚做的一层添加进去
    
    # 加一个断言，来验证输出的shape是否符合预期
    multiplier = 2 ** (num_subsampling - 1)
    print(layers[-1].get_shape().as_list()[1:])  # 打印查看  8*8*128
    assert layers[-1].get_shape().as_list()[1:] \
        == [input_size[0] / multiplier,
            input_size[1] / multiplier,
            num_filter_base * multiplier]  
    with tf.variable_scope('fc'):  # 最后是全连接层
        # layer[-1].shape : [None, width, height, channel]
        # kernal_size: image_width, image_height
        # 最初是sample*8*8*128,得到的是 sample*128
        global_pool = tf.reduce_mean(layers[-1], [1,2])  
        logits = tf.layers.dense(global_pool, class_num)
        layers.append(logits)
    return layers[-1]

# 下面4行的输入和之前是一样的
x = tf.placeholder(tf.float32, [None, 3072])
y = tf.placeholder(tf.int64, [None])
# [None], eg: [0,5,6,3]
x_image = tf.reshape(x, [-1, 3, 32, 32])
# 再进行轴滚动
x_image = tf.transpose(x_image, perm=[0, 2, 3, 1])
# 得到输出
y_ = res_net(x_image, [2,3,2], 32, 10)

# 下面内容和之前没差别
loss = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=y_)
# y_ -> sofmax
# y -> one_hot
# loss = ylogy_

# indices
predict = tf.argmax(y_, 1)
# [1,0,1,1,1,0,0,0]
correct_prediction = tf.equal(predict, y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))

with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.average_pooling2d instead.
[8, 8, 128]
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.


In [4]:
# 训练这部分和之前一致的
init = tf.global_variables_initializer() 
batch_size = 20
train_steps = 10000
test_steps = 100

# train 10000次 76.55%  为什么没有之前好，因为我们没有加入bn等一些扩展，但是我们这次没加，
# 通过fine-tune，得到94%
with tf.Session() as sess:
    sess.run(init)
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        loss_val, acc_val, _ = sess.run(
            [loss, accuracy, train_op],
            feed_dict={
                x: batch_data,
                y: batch_labels})
        if (i+1) % 100 == 0:
            print('[Train] Step: %d, loss: %4.5f, acc: %4.5f' 
                  % (i+1, loss_val, acc_val))
        if (i+1) % 1000 == 0:
            test_data = CifarData(test_filenames, False)
            all_test_acc_val = []
            for j in range(test_steps):
                test_batch_data, test_batch_labels \
                    = test_data.next_batch(batch_size)
                test_acc_val = sess.run(
                    [accuracy],
                    feed_dict = {
                        x: test_batch_data, 
                        y: test_batch_labels
                    })
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            print('[Test ] Step: %d, acc: %4.5f' % (i+1, test_acc))  

[Train] Step: 100, loss: 1.98531, acc: 0.35000
[Train] Step: 200, loss: 1.84638, acc: 0.25000
[Train] Step: 300, loss: 1.94834, acc: 0.25000
[Train] Step: 400, loss: 1.61439, acc: 0.60000
[Train] Step: 500, loss: 1.52786, acc: 0.45000
[Train] Step: 600, loss: 1.87887, acc: 0.20000
[Train] Step: 700, loss: 1.25330, acc: 0.50000
[Train] Step: 800, loss: 1.40641, acc: 0.40000
[Train] Step: 900, loss: 1.33005, acc: 0.55000
[Train] Step: 1000, loss: 1.59547, acc: 0.40000
(10000, 3072)
(10000,)
[Test ] Step: 1000, acc: 0.42850
[Train] Step: 1100, loss: 1.29622, acc: 0.50000
[Train] Step: 1200, loss: 1.39813, acc: 0.65000
[Train] Step: 1300, loss: 1.72328, acc: 0.65000
[Train] Step: 1400, loss: 1.40228, acc: 0.55000
[Train] Step: 1500, loss: 1.54440, acc: 0.55000
[Train] Step: 1600, loss: 1.55930, acc: 0.40000
[Train] Step: 1700, loss: 1.35021, acc: 0.55000
[Train] Step: 1800, loss: 1.35498, acc: 0.35000
[Train] Step: 1900, loss: 1.14276, acc: 0.55000
[Train] Step: 2000, loss: 1.25365, acc: 0